In [19]:
!jupyter nbconvert --to script 'graph_classification_tutorial.ipynb'

[NbConvertApp] Converting notebook graph_classification_tutorial.ipynb to script
[NbConvertApp] Writing 4217 bytes to graph_classification_tutorial.py


In [14]:
!pip3 install pandas==0.24 --user
!pip3 install botocore==1.12.250 --user

You are using pip version 19.0.2, however version 19.2.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 19.0.2, however version 19.2.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [26]:
%%sh
default_docker_name="dgllib/dgl-sagemaker-gpu:dgl_0.4_pytorch_1.2.0_rdkit"
docker pull $default_docker_name

docker_name="sagemaker-graph-class"

account=$(aws sts get-caller-identity --query Account --output text)
echo $account
region=$(aws configure get region)
region=${region:-us-east-2}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${docker_name}:latest"
# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${docker_name}" > /dev/null 2>&1
if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${docker_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
$(aws ecr get-login --region ${region} --no-include-email)

docker tag ${default_docker_name} ${fullname}

docker push ${fullname}

dgl_0.4_pytorch_1.2.0_rdkit: Pulling from dgllib/dgl-sagemaker-gpu
35c102085707: Already exists
251f5509d51d: Already exists
8e829fe70a46: Already exists
6001e1789921: Already exists
9f0a21d58e5d: Already exists
8810fcda1e6e: Already exists
d701a76e3193: Already exists
34be232fb7a6: Already exists
7e62b1ed3410: Already exists
47526c6630b9: Already exists
aed822d054ca: Already exists
4755ed5bed2d: Already exists
e08ece828d6d: Already exists
21f7e5dd23c0: Already exists
8385064a1256: Already exists
4a916ee6dd88: Already exists
5224460d79b7: Already exists
969f958f63e3: Already exists
031b90ba4752: Already exists
d9075a74f235: Already exists
60019ab50995: Already exists
5225800ad4f4: Already exists
5b628ec4dfa6: Already exists
84da67d22223: Already exists
1d090cebf694: Already exists
50d54af9151f: Already exists
a4cb5f37600b: Pulling fs layer
a4cb5f37600b: Verifying Checksum
a4cb5f37600b: Download complete
a4cb5f37600b: Pull complete
Digest: sha256:ada45f641e5b3ad529900f458e44c78b0934d1d1

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



In [27]:
import boto3
import re

import os
import numpy as np
from sagemaker import get_execution_role

import sagemaker as sage
from time import gmtime, strftime

In [28]:
# S3 prefix
prefix = 'dgl_example'

# Setup session
sess = sage.Session()

# Get Sagemaker role
role = 'arn:aws:iam::397262719838:role/service-role/AmazonSageMaker-ExecutionRole-20171213T134005'

CODE_PATH = 'graph_classification_tutorial.py'
code_location = sess.upload_data(CODE_PATH, key_prefix=prefix)

account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name
image = '{}.dkr.ecr.{}.amazonaws.com/sagemaker-graph-class:latest'.format(account, region)
print(image)

params = {}
params['entrypoint'] = CODE_PATH
tree = sage.estimator.Estimator(image,
                       role, 1, 'ml.p3.2xlarge',
                       hyperparameters=params,
                       output_path="s3://dgl-internal-example/output".format(sess.default_bucket()),
                       sagemaker_session=sess)

data = {}
data['training-code'] = code_location
data['training-data'] = 's3://dgl-sagemaker-dataset/dgl/'

tree.fit(data)

397262719838.dkr.ecr.us-east-2.amazonaws.com/sagemaker-graph-class:latest
2019-10-17 10:05:29 Starting - Starting the training job...
2019-10-17 10:05:31 Starting - Launching requested ML instances...
2019-10-17 10:06:25 Starting - Preparing the instances for training.........
2019-10-17 10:07:39 Downloading - Downloading input data
2019-10-17 10:07:39 Training - Downloading the training image............../opt/ml/input/data/training-code /
graph_classification_tutorial.py:52: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  dataset[i][0].ndata['feat']= torch.tensor(dataset[i][0].ndata['feat']).float()
Extracting file to /root/.dgl/tu_ENZYMES
DGLGraph(num_nodes=37, num_edges=168,
         ndata_schemes={'feat': Scheme(shape=(18,), dtype=torch.float32)}
         edata_schemes={})
tensor(5)
Traceback (most recent call last):
  File "graph_cl